# Content Words

In [ ]:
!apt-get -qq install -y libarchive-dev && pip install -U libarchive
import libarchive

Selecting previously unselected package libarchive-dev:amd64.
(Reading database ... 160983 files and directories currently installed.)
Preparing to unpack .../libarchive-dev_3.2.2-3.1ubuntu0.6_amd64.deb ...
Unpacking libarchive-dev:amd64 (3.2.2-3.1ubuntu0.6) ...
Setting up libarchive-dev:amd64 (3.2.2-3.1ubuntu0.6) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
  Created wheel for libarchive: filename=libarchive-0.4.7-cp27-none-any.whl size=31634 sha256=7eefd4047f5a35256e9870a9881826bc4270fd8caa196ee742191602839b5bc8
  Stored in directory: /root/.cache/pip/wheels/3b/5c/fa/92ee330d259e8fa5bedbd53f67040710fe81cfa463b8711d26
Successfully built libarchive


In [17]:
!pip install spacy
!pip install textacy

In [13]:
!pip install functoolz

  Created wheel for functoolz: filename=functoolz-0.4-cp27-none-any.whl size=3069 sha256=188919abfd32a42dc8fa5eee41eb2321be2ceab15264186acd6b4a9924bf3756
  Stored in directory: /root/.cache/pip/wheels/84/6b/7b/030b42288da092e557bfd48baf003daa5ac4dc18f53f21cd7d
Successfully built functoolz


In [27]:
import functoolz 

import textacy


NameError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
% matplotlib inline

from __future__ import division

import numpy as np 
import pandas as pd 

import spacy
from spacy.tokens.doc import Doc
import inspect
import textacy
from textacy.vsm import Vectorizer
import textacy.vsm

import scipy.sparse as sp

from tqdm import *

import re

NameError: ignored

Loading the data

In [ ]:
tweets = pd.read_csv('tweet_ids/2015_Nepal_Earthquake_en/stripped_filled_tweets.csv', encoding = 'ISO-8859-1')

In [ ]:
tweets = pd.read_csv('/content/drive/My Drive/data/filled_tweets.csv', encoding = 'ISO-8859-1')

In [ ]:
tweets.head()

,Unnamed: 0,label,tweet_id,tweet_texts
0,1,infrastructure_and_utilities_damage,'591902695822331904',RT @DailySabah: #LATEST #Nepal's Kantipur TV s...
1,2,injured_or_dead_people,'591902695943843840',RT @iamsrk: May Allah look after all. Here r t...
2,3,missing_trapped_or_found_people,'591902696371724288',RT @RT_com: LATEST: 108 killed in 7.9-magnitud...
3,4,sympathy_and_emotional_support,'591902696375877632',RT @Edourdoo: Shocking picture of the earthqua...
4,5,sympathy_and_emotional_support,'591902696895950848',Indian Air Force is ready to help the people o...


In [ ]:
tweets.head()

,Unnamed: 0,label,tweet_id,tweet_texts
0,0,other_useful_information,'591902695562170368',NaN
1,1,infrastructure_and_utilities_damage,'591902695822331904',RT @DailySabah: #LATEST #Nepal's Kantipur TV s...
2,2,injured_or_dead_people,'591902695943843840',RT @iamsrk: May Allah look after all. Here r t...
3,3,missing_trapped_or_found_people,'591902696371724288',RT @RT_com: LATEST: 108 killed in 7.9-magnitud...
4,4,sympathy_and_emotional_support,'591902696375877632',RT @Edourdoo: Shocking picture of the earthqua...


In [ ]:
tweets = tweets.dropna()

## Preprocessing

For my tweets to be informative, there are a few terms I can immediately remove. For instance, any urls won't be useful to the rescue teams. Equally, any '@...' are just calling another twitter handle, and are equally not useful. 

In [ ]:
# removing URLS
tweets.tweet_texts = tweets.tweet_texts.apply(lambda x: re.sub(u'http\S+', u'', x))   

# removing @... 
tweets.tweet_texts = tweets.tweet_texts.apply(lambda x: re.sub(u'(\s)@\w+', u'', x))

# removing hashtags
tweets.tweet_texts = tweets.tweet_texts.apply(lambda x: re.sub(u'#', u'', x))

In [ ]:
tweets.tweet_texts.head()

0    RT: LATEST Nepal's Kantipur TV shows at least ...
1    RT: May Allah look after all. Here r the emerg...
2    RT: LATEST: 108 killed in 7.9-magnitude Nepal ...
3    RT: Shocking picture of the earthquake in Nepa...
4    Indian Air Force is ready to help the people o...
Name: tweet_texts, dtype: object

There are alot of `u'RT'` terms in the tweet texts. Since this isn't a word, SpaCy doesn't know how to handle them. Since these add nothing to the content of a tweet, I'm just going to get rid of them. 

In [ ]:
tweets.tweet_texts = tweets.tweet_texts.apply(lambda x: x.replace(u'RT', u''))

Tokenizing with SpaCy

In [ ]:
nlp = spacy.load('en')

In [ ]:
spacy_tweets = []

for doc in nlp.pipe(tweets.tweet_texts, n_threads = -1):
    spacy_tweets.append(doc)

In [ ]:
spacy_tweets[200]

MEA opens 24 hour Control Room for queries regarding the Nepal Earthquake.åÊ
Numbers:
+91 11 2301 2113
+91 11 2301 4104
+91 11 2301 7905

Not all tweets are equally useful. Some just contain prayers, such as

`Hope it doesn't rain. #Nepal`

whereas others are dense with useful information: 

`2 Dead, 100 Injured in Bangladesh From Nepal Quake`

How do I decide which parts of these tweets are most useful? One way to do it is to measure the term frequency-inverse document frequency (tf-idf) of each of the words in the corpus of tweets. This metric measures how important a word is in a corpus of tweets. 

## Getting the tf-idf values of content words. 

I can do a preliminary 'cleanup', by keeping only 'content words'. These are defined as : Numerals, Nouns and Verbs. Conveniantly, SpaCy has already organised this for us. 

In [ ]:
spacy_tweets[90]

NameError: ignored

I care most about tokens which are entities, and numbers. The other tokens have too much noise, so let's focus on these two:

In [ ]:
main_words = [u'earthquake', u'killed', u'injured', u'stranded', u'wounded', u'hurt', u'helpless', u'wrecked', u'nepal'] # get this from our dataset with target 1

In [ ]:
useful_entities = [u'NORP', u'FACILITY', u'ORG', u'GPE', u'LOC', u'EVENT', u'DATE', u'TIME']

In [ ]:
content_tweets = []
for single_tweet in tqdm(spacy_tweets):
    single_tweet_content = []
    for token in single_tweet: 
        if ((token.ent_type_ in useful_entities)  
            or (token.pos_ == u'NUM') 
            or (token.lower_ in main_words)):
            single_tweet_content.append(token)
    content_tweets.append(single_tweet_content)

NameError: ignored

In [ ]:
tweet_num = 200
print ("original_tweet \n" + str(spacy_tweets[tweet_num]) 
       + "\n\noriginal_tweet\n" + str([str(x) for x in spacy_tweets[tweet_num]])
       + "\n\ncontent_tweet\n" + str(content_tweets[tweet_num])
      )

So this has already gone some way to (crudely) isolating the interesting parts of a tweet. 

Unfortunately, SpaCy doesn't calculate tf-idf score automatically. There IS a library which can do this: [textacy](https://textacy.readthedocs.io/en/latest/index.html). Note: textacy is built on SpaCy.

I care about the tf-idf scores of the entire tweet, so will find the tf-idf score across the entire corpus of original tweets. 

In [ ]:
vectorizer = Vectorizer(weighting = 'tfidf')

To calculate the tf-idf score of all the tokens in the tweets, I can use `fit_transform()`. 

Note: I am using the `lemma_` attribute of each token, because tokens contain information about the documents. This means that 'Nepal' in the 100th tweet will have a different **token** from 'Nepal' in the 200th tweet, but the same `lemma__` attribute. This is what I want to compare - I don't want hundreds of 'Nepal' columns in my term matrix. 

In [ ]:
term_matrix = vectorizer.fit_transform([tok.lemma_ for tok in doc] for doc in spacy_tweets)

This matrix is a term-document matrix. What this means is that on top of having the tf-idf values, each row is a document (and each column is a word). 

If the tweet in row `i` contains the column in row `j`, then the element `matrix[i][j]` will contain the tf-idf value. If the tweet *doesn't* contan the word, the matrix value will be zero. 

In [ ]:
np_matrix = term_matrix.todense()

In [ ]:
np_matrix.shape

My ultimate goal is to create a dictionary, which maps from the tokens in the content tweets to some tf-idf score. To do this, I need to find out which tokens are at what columns in the term matrix. 

The vectorizer object has a dictionary, which maps token.lemma_ to its column. 

In [ ]:
for key in sorted(vectorizer.vocabulary)[1000:1015]:
    print key, vectorizer.vocabulary[key]

And each column (word) has a unique tf-idf value.

I can therefore map the value of the content tokens to their tf-idf, using the `vectorizer.vocabulary` dictionary. 

In [ ]:
for token in content_tweets[500]:
    print (token.lemma_, vectorizer.vocabulary[token.lemma_], np.max(np_matrix[:,vectorizer.vocabulary[token.lemma_]]))

In [ ]:
tfidf_dict = {}
content_vocab = []
for tweet in content_tweets: 
    for token in tweet: 
        if token.lemma_ not in tfidf_dict: 
            content_vocab.append(token.lemma_)
            tfidf_dict[token.lemma_] = np.max(np_matrix[:,vectorizer.vocabulary[token.lemma_]])

In [ ]:
for key in sorted(tfidf_dict)[500:505]:
    print ("WORD:" + str(key) + " -- tf-idf SCORE:" +  str(tfidf_dict[key]))

Success! 

## COntent Word-based Tweet Summarization (COWTS) 
As per [Rudra et al](http://dl.acm.org/citation.cfm?id=2806485). 

I'll be using [PyMathProg](http://pymprog.sourceforge.net/index.html) as my Integer Linear Programming Solver. This is a python interface for [GLPK](https://www.gnu.org/software/glpk/)

In [ ]:
from pymprog import *

ImportError: ignored

I want to maximize 
\begin{equation}
\sum_{i=1}^n x_{i} + \sum_{j = 1}^{m} Score(j) \cdot y_{j}
\end{equation}
Where $x_{i}$ is 1 if I include tweet i, or 0 if I don't, and where $y_{j}$ is 1 or 0 if each content word is included (and Score(j) is that word's tf-idf score). 

I'm going to subject this equation to the following constraints: 

1. 
\begin{equation}
\sum_{i=1}^{n} x_{i} \cdot Length(i) \leq L
\end{equation}
I want the total length of all the selected tweets to be less than some value L, which will be the length of my summary, L. I can vary L depending on how long I want my summary to be. 

2. 
\begin{equation}
\sum_{i \in T_{j}} x_{i} \geq y_{j}, j = [1,...,m]
\end{equation}
If I pick some content word $y_{j}$ (out of my $m$ possible content words) , then I want to have at least one tweet from the set of tweets which contain that content word, $T_{j}$. 

3. 
\begin{equation}
\sum_{j \in C_{i}} y_{j} \leq |C_{i}| \times x_{i}, i = [1,...,n]
\end{equation}
If I pick some tweet i (out of my $n$ possible tweets) , then all the content words in that tweet $C_{i}$ are also selected. 

In [ ]:
begin('COWTS')

NameError: ignored

In [ ]:
# Defining my first variable, x 
# This defines whether or not a tweet is selected
x = var('x', len(spacy_tweets), bool)

# Check this worked
x[1000]

In [ ]:
# Also defining the second variable, which defines
# whether or not a content word is chosen
y = var('y', len(content_vocab), bool)

In [ ]:
len(y), y[0]

Now that I have defined my variables, I can define the equation I am maximizing. 

In [ ]:
maximize(sum(x) + sum([tfidf_dict[content_vocab[j]]*y[j] for j in range(len(y))]));

Now, I can define my constraints. First, 
\begin{equation}
\sum_{i=1}^{n} x_{i} \cdot Length(i) \leq L
\end{equation}

In [ ]:
## Maximum length of the entire tweet summary

# Was 150 for the tweet summary, 
# But generated a 1000 word summary for CONABS
L = 1000

# hiding the output of this line since its a very long sum 
sum([x[i]*len(spacy_tweets[i]) for i in range(len(x))]) <= L;

These next two constraints are slightly more tricky, as I need a way to define which content words are in which tweets. 

However, the term matrix I defined using the vectorizer has all of this information. 

I'll begin by defining two helper methods

In [ ]:
def content_words(i):
    '''Given a tweet index i (for x[i]), this method will return the indices of the words in the 
    content_vocab[] array
    Note: these indices are the same as for the y variable
    '''
    tweet = spacy_tweets[i]
    content_indices = []
    
    for token in tweet:
        if token.lemma_ in content_vocab:
            content_indices.append(content_vocab.index(token.lemma_))
    return content_indices

In [ ]:
def tweets_with_content_words(j):
    '''Given the index j of some content word (for content_vocab[j] or y[j])
    this method will return the indices of all tweets which contain this content word
    '''
    content_word = content_vocab[j]
    
    index_in_term_matrix = vectorizer.vocabulary[content_word]
    
    matrix_column = np_matrix[:, index_in_term_matrix]
    
    return np.nonzero(matrix_column)[0]

I can now define the second constraint: 
\begin{equation}
\sum_{i \in T_{j}} x_{i} \geq y_{j}, j = [1,...,m]
\end{equation}

In [ ]:
for j in range(len(y)):
    sum([x[i] for i in tweets_with_content_words(j)])>= y[j]

And the third constraint:
\begin{equation}
\sum_{j \in C_{i}} y_{j} \leq |C_{i}| \times x_{i}, i = [1,...,n]
\end{equation}

In [ ]:
for i in range(len(x)):
    sum(y[j] for j in content_words(i)) >= len(content_words(i))*x[i]

In [ ]:
solve()

In [ ]:
result_x =  [value.primal for value in x]
result_y = [value.primal for value in y]

In [ ]:
end()

In [ ]:
chosen_tweets = np.nonzero(result_x)
chosen_words = np.nonzero(result_y)

In [ ]:
len(chosen_tweets[0]), len(chosen_words[0])

Lets take a look at the results! 

In [ ]:
for i in chosen_tweets[0]:
    print ('--------------')
    print spacy_tweets[i]

There is definitely noise amongst these tweets, but these tweets do successfully provide a good overview of the situation in Nepal.

I am going to compare this to random tweets, to make sure it does perform better than 16 randomly chosen tweets. 

In [ ]:
random_tweets = np.random.choice(spacy_tweets, size=11)

In [ ]:
for i in random_tweets:
    print ('--------')
    print i

A brief comparison does indicate that this method is far better than random choice at providing a situational overview. 

It's worth noting that even a random distribution will contain a fair amount of information, because of the selective nature in which we isolated tweets; this is already a subsample which contains a higher % of relevant information. 

In [ ]:
cowts_tweets = []
for i in chosen_tweets[0]:
    cowts_tweets.append(spacy_tweets[i])

Lets take a look at the first few tweets

In [ ]:
for tweet in cowts_tweets[:10]:
    print ('--------')
    print tweet

This notebook is getting long, so I'm going to save these tweets (which I will continue using) and start a fresh notebook for the next steps. 

## Saving everything for a fresh notebook 

In [ ]:
cowts_unicode = [x.text for x in cowts_tweets]

In [ ]:
cowts_dataframe = pd.DataFrame(cowts_unicode)

In [ ]:
cowts_dataframe.head()

,0
0,: LATEST Nepal's Kantipur TV shows at least 21...
1,Prayers for the affected people across SouthAs...
2,: Due to bulding collaps 12 People died in Eas...
3,": M7.9 - 29km ESE of Lamjung, Nepal 20 00 29..."
4,"Earthquake: 2015-04-25 17:30HKT M5.0 [28.0N,85..."


Saving it to a pickle: 

In [ ]:
cowts_dataframe.to_pickle('cowts_tweets.pkl')

In [ ]:
np.save('term_matrix.npy', np_matrix)

In [ ]:
np.save('tweet_indices.npy', chosen_tweets)

In [ ]:
np.save('vocab_to_idx.npy', vectorizer.vocabulary)

In [ ]:
np.save('content_vocab.npy', content_vocab)

In [ ]:
np.save('tfidf_dict.npy', tfidf_dict)